<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/Catalogue_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
cat = pd.read_json('/content/drive/My Drive/Library_catalogue_preprocessed.jl', lines=True)

In [3]:
cat

,title,author,storage,WD_signature,publisher,year,pages,language,tokens
0,Pomiary fizyczne za pomocą komputera : dla stu...,"Jagiełło, Michał (1941-2016).",magazyn,QC39 .P58 1999,Wydaw. Naukowe UAM,9999,238,pl,None
1,In my grandfather's footsteps : a Rabbi's note...,"Baumol, Avi.",magazyn,,Wydawnictwo Austeria Klezmerhojs,5779,274,en,None
2,[Senior Thai research Fund researches meet Jun...,Samnakngān Kōngthun Sanapsanun Kānwičhai.,magazyn,KJE4170 .A965 2011,Commission on Higher Education,2550,486,en,None
3,[Senior Thai research Fund researches meet Jun...,Samnakngān Kōngthun Sanapsanun Kānwičhai.,magazyn,KJE4170 .A965 2011,Commission on Higher Education,2550,171,en,None
4,Małżeństwo i rodzina w procesie przemian sekul...,"Zbyrad, Teresa.",magazyn,BX2351 .Z35 2020,Wydawnictwo Naukowe Uniwersytetu Pedagogicznego,2020,276,pl,None
...,...,...,...,...,...,...,...,...,...
1312052,Albertvs Z Woyny.,"Speelberg, Femke.",magazyn,,W Drukarniey v Dziedzicow Jakuba Sybeneichera,1613,4,en,"[albertvs, z, woyny]"
1312053,The Landgrave of Hessen his princelie receiuin...,"Monings, Edward ( -1602).",magazyn,PN4759 .A48 2007,by Robert Robinson,1596,27,en,"[landgrave, hessen, princelie, receiuing, maie..."
1312054,The scholemaster or plaine and perfite way of ...,"Ascham, Roger (1515-1568).",magazyn,PJ5060.P58 P64 1986,printed by John Daye,1571,67,en,"[scholemaster, plaine, perfite, way, teach, ch..."
1312055,"The image of Old Testament : lately expressed,...",,,,print. by Io. Frellon,1549,4,en,"[image, old, testament, lately, express, set, ..."
